# 2 分词器
## 2.1 Unicode 标准

In [ ]:
print(ord('s'))

print(ord('牛'))

115
29275


In [8]:
print(chr(115))
print(chr(29275))

s
牛


>### 问题 (1 分) 
>* (a) chr(0)返回什么Unicode字符？
>* (b) 该字符的字符串表示与打印表示有何不同？
>* (c) 当该字符出现在文本中会发生什么？

In [ ]:
chr(0)#'\x00'

'\x00'

In [21]:
l = "this is a test" + chr(0) + "string"

print(l)

this is a test string


## 2.2 Unicode 编码

In [34]:
test_string ="Hello world 牛逼😭"
utf_8_test_string = test_string.encode('utf-8')
print(utf_8_test_string)
print(type(utf_8_test_string))

#拆解字符
print(list(utf_8_test_string))
print("字符长度:", len(test_string), len(utf_8_test_string))


#解码
print(utf_8_test_string.decode('utf-8'))

b'Hello world \xe7\x89\x9b\xe9\x80\xbc\xf0\x9f\x98\xad'
<class 'bytes'>
[72, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100, 32, 231, 137, 155, 233, 128, 188, 240, 159, 152, 173]
字符长度: 15 22
Hello world 牛逼😭


> ### 问题 (3 分)
> * (a) 为什么我们更倾向于在UTF-8编码的字节上训练分词器，而不是UTF-16或UTF-32？比较这些编码对不同输入字符串的输出可能有所帮助。
> 
> 交付要求：一至两句话的回答。
> * (b) 考虑以下（错误的）函数，其目的是将UTF-8字节串解码为Unicode字符串。为什么这个函数是错误的？
> ```
>def decode_utf8_bytes_to_str_wrong(bytestring: bytes):  # 注意参数名修正！    
>    return "".join([bytes([b]).decode("utf-8") for b in bytestring])
>```
> 提供一个会产生错误结果的输入字节串示例。


In [4]:
test_string ="Hello world 牛逼😭"
print(test_string.encode('utf-8'))
print(test_string.encode('utf-16'))
print(test_string.encode('utf-32'))

b'Hello world \xe7\x89\x9b\xe9\x80\xbc\xf0\x9f\x98\xad'
b'\xff\xfeH\x00e\x00l\x00l\x00o\x00 \x00w\x00o\x00r\x00l\x00d\x00 \x00[r<\x90=\xd8-\xde'
b'\xff\xfe\x00\x00H\x00\x00\x00e\x00\x00\x00l\x00\x00\x00l\x00\x00\x00o\x00\x00\x00 \x00\x00\x00w\x00\x00\x00o\x00\x00\x00r\x00\x00\x00l\x00\x00\x00d\x00\x00\x00 \x00\x00\x00[r\x00\x00<\x90\x00\x00-\xf6\x01\x00'


In [ ]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):  # 注意参数名修正！    
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])

test_string ="Hello world 牛逼😭"
utf_8_test_string = test_string.encode('utf-8')
print(decode_utf8_bytes_to_str_wrong(utf_8_test_string))

#UTF-8特性‌：多字节字符（如中文字符）由多个连续字节组成（如中文字符通常占用3字节）。
# 逐字节解码会将这些字节视为独立字符，导致解码失败（UnicodeDecodeError）。

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe7 in position 0: unexpected end of data

In [10]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):  # 注意参数名修正！    
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])

test_string ="Hello world"
utf_8_test_string = test_string.encode('utf-8')
print(decode_utf8_bytes_to_str_wrong(utf_8_test_string))

Hello world


## 2.3 子字标记化（subword tokenization）
子字标记化（subword tokenization）则介于单词级和字节级之间，提供了一种折中方案。与仅有256个条目的字节级词汇表不同，子词标记器通过扩大词汇量来更高效地压缩原始字节序列。其核心思想是：如果某些字节序列（如 b'the'）在训练数据中频繁出现，就将其合并为一个单独的标记，从而将原本多个字节组成的序列压缩为一个单元。这样既能保持对罕见词和未知词的处理能力，又能显著缩短平均序列长度。

如何选择这些子词单元？Sennrich 等人（2016）提出采用字节对编码（Byte Pair Encoding, BPE），这是一种源自数据压缩技术的算法。BPE 通过迭代地查找并合并出现频率最高的相邻字节对，逐步构建出一组高效的子词单元。每次合并都会引入一个新的符号来代表该字节对，并将其加入词汇表。这一过程持续进行，直到达到预设的词汇表大小。

由于BPE优先合并高频模式，因此最终的词汇表能最大程度地提升整体压缩效率——常见词或词片段更可能被表示为单一标记。在本任务中，我们将实现一种基于字节的BPE分词器，其词汇项由原始字节及其合并后的序列表示。这种方法结合了字节级分词器的鲁棒性与子词级的高效性，在处理未登录词的同时保持合理的序列长度。整个构建词汇表的过程也被称为“训练”BPE分词器，是实现高效文本表示的关键步骤。